In [ ]:
import segno
from PIL import Image, ImageChops, ImageDraw, ImageFont
import json
import io

ppi = 200
inch_per_cm = 0.393701

cm_to_px = lambda x: int(x*ppi*inch_per_cm)


In [36]:

def create_qr(data, height=1):
    qrcode = segno.make(data, boost_error=False, micro=False)
    out = io.BytesIO()
    qrcode.save(out, scale=height, border=0, kind='png')
    out.seek(0)
    local_img = Image.open(out)
    print(local_img)
    return local_img
    

def save_img(img, path, printer:str="joost_thuis"):
    with open(f'./printer_data/{printer}.json', 'r') as f:
        printer_data = json.load(f)

    offset_img = ImageChops.offset(
        img, 
        cm_to_px(-printer_data["MARGIN_LEFT"]), 
        cm_to_px(-printer_data["MARGIN_TOP"])
    )

    offset_img.save(path)
    


# student: id, test: exam id, question id


In [37]:

def create_answer_box(img, y1, height, qr_code_text='q123456'):
    offset = 1.5
    
    qr_code = create_qr(qr_code_text, 1)
    qr_size = cm_to_px(1.5)

    
    p1 = (cm_to_px(offset)-4, cm_to_px(offset))
    p1_rect = (cm_to_px(offset)+qr_size, cm_to_px(offset))
    p2 = img.size[0] - cm_to_px(offset+1) + 4, cm_to_px(offset+height)
    p2_rect = img.size[0] - cm_to_px(offset+1), cm_to_px(offset+height) + qr_size

    qr_code = qr_code.resize((qr_size, qr_size))

    img.paste(qr_code, p1)
    img.paste(qr_code, p2)
    
    draw = ImageDraw.Draw(img)
    draw.rectangle([p1_rect, p2_rect], width=5, outline='black')
    
    
    
    return img

img = Image.new('RGB', (int(8.25*ppi), int(11.75*ppi)), color='white')

for i in range(3):
    create_answer_box(img, cm_to_px(i*5), 5)



save_img(img, 'output.png')




<PIL.PngImagePlugin.PngImageFile image mode=1 size=21x21 at 0x18E1880D990>
<PIL.PngImagePlugin.PngImageFile image mode=1 size=21x21 at 0x18E1880D780>
<PIL.PngImagePlugin.PngImageFile image mode=1 size=21x21 at 0x18E1880DED0>


In [45]:
from PIL import Image, ImageFont, ImageDraw
import cv2
import numpy as np
from pyzbar.pyzbar import decode

def scan_qrcode_from_image_path(pillow_image):
    # Load the image from file using Pillow
    pillow_image = Image.open(image_path)
    

    # Ensure the image is in RGB mode (in case it's RGBA or grayscale)
    if pillow_image.mode != 'RGB':
        pillow_image = pillow_image.convert('RGB')
    
    # Convert Pillow image to OpenCV format (numpy array)
    open_cv_image = np.array(pillow_image)

    # Convert RGB to BGR (OpenCV uses BGR format)
    open_cv_image = open_cv_image[:, :, ::-1]

    # Decode QR codes
    qr_codes = decode(open_cv_image)

    # Create a drawing context
    draw = ImageDraw.Draw(pillow_image)

    # Iterate through detected QR codes
    for qr in qr_codes:
        # Get the data from the QR code
        qr_data = qr.data.decode('utf-8')
        print(f"QR Code Data: {qr_data} {qr.polygon}")
        

        # Get the bounding box (rectangle)
        rect_points = qr.rect  # This is a tuple (x, y, width, height)
        top_left = (rect_points[0], rect_points[1])
        bottom_right = (rect_points[0] + rect_points[2], rect_points[1] + rect_points[3])

        # Draw the rectangle (color: green, thickness: 3)
        draw.rectangle([top_left, bottom_right], outline="green", width=3)
        
        # Display the decoded QR code data on the image
        font = ImageFont.load_default()
        text_position = (top_left[0], top_left[1] - 10)  # Place text above the rectangle
        draw.text(text_position, qr_data, fill="green", font=font)
        

    # Show the image with the QR code and bounding box
    pillow_image.show()

# Specify the path to your image file
image_path = "output.png"
pillow_image = Image.open(image_path)

# Scan QR codes from the image file
scan_qrcode_from_image_path(pillow_image)


QR Code Data: q123456 [Point(x=1427, y=464), Point(x=1427, y=581), Point(x=1544, y=581), Point(x=1544, y=464)]
QR Code Data: q123456 [Point(x=83, y=71), Point(x=83, y=188), Point(x=200, y=188), Point(x=200, y=71)]
